In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv, find_dotenv
import PyPDF2
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.question_answering import load_qa_chain
from langserve import RemoteRunnable

_ = load_dotenv(find_dotenv())

chat_model = AzureChatOpenAI(
    openai_api_version="2023-09-01-preview",
    azure_endpoint=os.getenv('AZURE_API_ENDPOINT'),
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    azure_deployment=os.getenv('OPENAI_DEPLOYMENT_NAME'),
    model_name=os.getenv('OPENAI_MODEL_NAME'),
    model_version=os.getenv('OPENAI_API_VERSION')
)

In [ ]:
from langchain.chains.api import open_meteo_docs

# Création de la chaine météo
Meteochain = APIChain.from_llm_and_api_docs(
    llm,
    open_meteo_docs.OPEN_METEO_DOCS,
    verbose=True,
    limit_to_domains=["https://api.open-meteo.com/"],
)

def askWeatherQuestion(weatherQuestion):
    response = Meteochain.invoke(weatherQuestion)["output"]
    return response

In [ ]:
# Documentation sur l'API TMDB
moovie_api_docs = """
API documentation:
Base URL: (https://api.themoviedb.org)
L'API de The Movie Database (TMDB) fournit l'accès à une variété de endpoint, chacun servant différentes données liées aux films, aux émissions de télévision, et plus encore.
Répond toujours en francais 
"""

tmdbKey = os.getenv("tmdb_key")
# Nécéssaire pour utiliser l'API
headers = {
    "Authorization": f"Bearer {tmdbKey}" 
}

# Création de la chaine
moovie_api_chain = APIChain.from_llm_and_api_docs(
    llm,
    moovie_api_docs,
    verbose=True,
    limit_to_domains=["https://api.themoviedb.org"],
    headers=headers,
    memory=ConversationBufferMemory(),
)

def AskTMDB(question):
    reponse = moovie_api_chain.invoke(question)["output"]
    return reponse